In [1]:
%load_ext autoreload
%autoreload 2


In [31]:
from data import get_track
from IPython.display import clear_output
import tensorflow as tf 
import numpy as np

In [2]:
trk = get_track()
clear_output()

In [5]:
pidx = trk.index.get_level_values(0).unique()

In [14]:
from data import padBatch, get_batch

In [17]:
df = trk.loc[pidx[4]]
trL = df.index.max()+1

In [35]:
padding = tf.constant([[0,25-int(trL)],[0,0]])

res =  tf.pad(df[0*int(trL):1*int(trL)].astype(np.float32)    ,padding,"CONSTANT")


In [67]:
import numpy as np
import tensorflow as tf


In [71]:
beids = np.random.choice(trk.index.get_level_values(0).unique(),5)
trL = trk.loc[beids[0]].posZ.max()+1
trk.loc[beids[0]].index.size/trL

21.0

In [137]:
trk.posZ.max()

22

In [77]:
MAX_LAYER = 25


In [132]:
def padBatch(df):
    trL = df.posZ.max()+1
    numParticles = int(df.index.size/trL)
    
    padding = tf.constant([[0,MAX_LAYER-trL],[0,0]])
    res =  tf.pad(df[0*trL:1*trL],padding,"CONSTANT")
    for pidx in range(1,numParticles-1):
        padding = tf.constant([[0,MAX_LAYER-trL],[0,0]])
        r1 = tf.pad(df[pidx*trL:(pidx+1)*trL],padding,"CONSTANT")
        res = tf.concat([res,r1],0) 
    return res 
padded = padBatch(trk.loc[beids[0]])


In [138]:
padded.shape[0]

500

In [88]:
t1 = tf.constant([[1,2],[3,4]])
padding = tf.constant([[1,2],[1,2]])
tf.pad(t1,padding,"CONSTANT")

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[0, 0, 0, 0, 0],
       [0, 1, 2, 0, 0],
       [0, 3, 4, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])>

In [47]:
trk.loc[beids[4]].posZ.max()

21

In [55]:
eid = trk.index.get_level_values(0).unique()
tid = trk.index.get_level_values(1).unique()
trk.loc[eid[2],tid[0]]

dX        dY        dZ       Ekine      posX      posY  \
EventID TrackID                                                                 
3       1        0.014982  0.011148  0.999826  148.987717 -4.124664  2.281091   
        1        0.014982  0.011148  0.999826  148.987717 -3.397784  3.169025   
        1        0.014982  0.011148  0.999826  148.987717 -2.613147  3.812561   
        1        0.014982  0.011148  0.999826  148.987717 -2.452003  3.938205   
        1        0.014982  0.011148  0.999826  148.987717 -2.242367  4.079671   
...                   ...       ...       ...         ...       ...       ...   
        1        0.054807 -0.033875  0.997922    9.251559  0.390218  5.989302   
        1        0.054807 -0.033875  0.997922    9.251559  0.458060  5.917573   
        1        0.054807 -0.033875  0.997922    9.251559  0.589257  5.817865   
        1        0.054807 -0.033875  0.997922    9.251559  0.872694  5.810639   
        1        0.054807 -0.033875  0.997922    9.251559  1.265330  5.869817   

                 posZ  
EventID TrackID        
3       1           0  
        1           1  
        1           2  
        1           3  
        1           4  
...               ...  
        1          16  
        1          17  
        1          18  
        1          19  
        1          20  

[462 rows x 7 columns]

# Problems
hit (eventID + trackID) != allhit(eventID + trackID)